In [2]:
import os
import requests
import zstandard as zstd
from tqdm import tqdm
import pandas as pd
import random
import shutil
import rasterio
from rasterio.warp import calculate_default_transform
from rasterio.warp import reproject, Resampling
import numpy as np
from PIL import Image
import time

In [3]:
dataset_dir = r'C:\Users\isaac\Desktop\BigEarthTests\OnePBigEarthNetCopy'
subset_dir = r'C:\Users\isaac\Desktop\BigEarthTests\Subsets'
metadata_file = r'C:\Users\isaac\Downloads\metadata.parquet'
unwanted_metadata_file = r'C:\Users\isaac\Downloads\metadata_for_patches_with_snow_cloud_or_shadow.parquet'
    
metadata_df = pd.read_parquet(metadata_file)
snow_cloud_metadata_df = pd.read_parquet(unwanted_metadata_file)

In [ ]:
def resizeTiffFiles(input_tiff, output_tiff, new_width, new_height):
    with rasterio.open(input_tiff) as src:
        transform, width, height = calculate_default_transform(src.crs, src.crs, new_width, new_height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': src.crs,
            'transform': transform,
            'width': new_width,
            'height': new_height
        })
        with rasterio.open(output_tiff, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=src.crs,
                    resampling=Resampling.nearest
                )

In [ ]:
def resize_tiff_file(input_tiff, output_tiff, new_width=120, new_height=120):
    """Resize a single TIFF file."""
    with rasterio.open(input_tiff) as src:
        transform, width, height = calculate_default_transform(
            src.crs, src.crs, new_width, new_height, *src.bounds)
        
        # Update metadata for new width and height
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': src.crs,
            'transform': transform,
            'width': new_width,
            'height': new_height
        })
        
        # Write directly to the output file
        with rasterio.open(output_tiff, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=src.crs,
                    resampling=Resampling.nearest
                )

In [44]:
def createSubsets(dataset_dir, subsets, metadata_df, subset_dir):
    metadata_50 = pd.DataFrame(columns=metadata_df.columns)
    metadata_10 = pd.DataFrame(columns=metadata_df.columns)
    metadata_1 = pd.DataFrame(columns=metadata_df.columns)

    for subset in subsets.values():
        if not os.path.exists(subset):
            os.makedirs(subset)
            print(f"Created subset: {subset}")
        else:
            print(f"Subset already exists: {subset}")
    
    for folder in tqdm(os.listdir(dataset_dir), desc='Creating subsets'):
        folder_path = os.path.join(dataset_dir, folder)

        if os.path.isdir(folder_path):
            # List all subfolders
            subfolders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]

            # Calculate number of folders in each subset
            num_subfolders = len(subfolders)
            num_50_percent = min(max(1, num_subfolders // 2), num_subfolders)
            num_10_percent = min(max(1, num_subfolders // 10), num_subfolders)
            num_1_percent = min(max(1, num_subfolders // 100), num_subfolders)

            # Select random subfolders for each subset
            selected_50 = random.sample(subfolders, num_50_percent)
            selected_10 = random.sample(subfolders, num_10_percent)
            selected_1 = random.sample(subfolders, num_1_percent)

            # Filter metadata for each subset
            metadata_50 = pd.concat([metadata_50, metadata_df[metadata_df['patch_id'].isin(selected_50)]])
            metadata_10 = pd.concat([metadata_10, metadata_df[metadata_df['patch_id'].isin(selected_10)]])
            metadata_1 = pd.concat([metadata_1, metadata_df[metadata_df['patch_id'].isin(selected_1)]])

            # Copy selected subfolders to each subset directory
            for selected in selected_50:
                dest_path = os.path.join(subsets['50%'], folder, selected)
                if not os.path.exists(dest_path):  # Check if the destination folder exists
                    try:
                        shutil.copytree(os.path.join(folder_path, selected), dest_path)
                    except FileExistsError:
                        print(f"Directory already exists, skipping: {dest_path}")

            for selected in selected_10:
                dest_path = os.path.join(subsets['10%'], folder, selected)
                if not os.path.exists(dest_path):  # Check if the destination folder exists
                    try:
                        shutil.copytree(os.path.join(folder_path, selected), dest_path)
                    except FileExistsError:
                        print(f"Directory already exists, skipping: {dest_path}")
            
            for selected in selected_1:
                dest_path = os.path.join(subsets['1%'], folder, selected)
                if not os.path.exists(dest_path):  # Check if the destination folder exists
                    try:
                        shutil.copytree(os.path.join(folder_path, selected), dest_path)
                    except FileExistsError:
                        print(f"Directory already exists, skipping: {dest_path}")

    # Save metadata for each subset
    metadata_50.to_csv(os.path.join(subset_dir, 'metadata_50_percent.csv'), index=False)
    metadata_10.to_csv(os.path.join(subset_dir, 'metadata_10_percent.csv'), index=False)
    metadata_1.to_csv(os.path.join(subset_dir, 'metadata_1_percent.csv'), index=False)

In [45]:
def count_subfolders(base_dir, folder):
    # Dictionary to hold folder counts
    folder_counts = {}
    total_subfolders = 0  # Initialize total subfolder counter
    
    # Iterate through all folders in the base directory
    for folder in tqdm(os.listdir(base_dir), desc="Processing folders"):
        folder_path = os.path.join(base_dir, folder)
        
        # Check if the current path is a directory
        if os.path.isdir(folder_path):
            # Count subdirectories within this folder
            subfolder_count = sum(os.path.isdir(os.path.join(folder_path, subfolder)) for subfolder in os.listdir(folder_path))
            folder_counts[folder] = subfolder_count
        
            # Update total subfolder count
            total_subfolders += subfolder_count

    # Print total subfolders
    return total_subfolders, folder

# Function to calculate and display subfolder count and percentage
def display_percentage(partial_count, full_count, folder_name):
    percentage = (partial_count / full_count) * 100
    print(f"Folder: {folder_name} | Subfolder Count: {partial_count} | Percentage: {percentage:.2f}%")

def labels_to_binary_vector(labels, unique_labels):
    binary_vector = [1 if label in labels else 0 for label in unique_labels]
    return binary_vector


In [46]:
# Function to preprocess the BigEarthNet dataset
def BigEarthNetDataPreprocessing(dataset_dir, subset_dir, metadata_df, snow_cloud_metadata_df):
    # Stage 3: Create subsets of the dataset and filter metadata
    subsets = {
        '50%': os.path.join(subset_dir, '50%'),
        '10%': os.path.join(subset_dir, '10%'),
        '1%': os.path.join(subset_dir, '1%')
    }
    metadata_df = pd.read_parquet(metadata_file)

    if os.path.exists(subsets['50%']) and os.path.exists(subsets['10%']) and os.path.exists(subsets['1%']):
        print('Subsets already exist')
    else:
        createSubsets(dataset_dir, subsets, metadata_df, subset_dir)

    full_subfolder_count, folder = count_subfolders(dataset_dir, '100%BigEarthNet')
    half_subfolder_count, folder = count_subfolders(subsets['50%'], '50%BigEarthNet' )
    tenth_subfolder_count, folder = count_subfolders(subsets['10%'], '10%BigEarthNet' )
    hundredth_subfolder_count, folder = count_subfolders(subsets['1%'], '1%BigEarthNet')

    # Display the counts and percentages for each folder
    print(f"Total subfolder count in full dataset: {full_subfolder_count}\n")
    display_percentage(half_subfolder_count, full_subfolder_count, '50%BigEarthNet')
    display_percentage(tenth_subfolder_count, full_subfolder_count, '10%BigEarthNet')
    display_percentage(hundredth_subfolder_count, full_subfolder_count, '1%BigEarthNet')

    # Stage 4: Add a binary vector to the metadata files to indicate the presence of a specific land cover class
    unique_labels = metadata_df['labels'].explode().unique()
    print(unique_labels)

    metadata_df['binary_vector'] = metadata_df['labels'].apply(lambda x: labels_to_binary_vector(x, unique_labels))

    print(metadata_df.columns, "\n")

    # Save the updated DataFrame to a new Parquet file
    updated_metadata_file = r'C:\Users\isaac\Downloads\updated_metadata.parquet'
    metadata_df.to_parquet(updated_metadata_file)

    updated_metadata_df = pd.read_parquet(updated_metadata_file)
    print(updated_metadata_df.columns, "\n")

In [47]:
BigEarthNetDataPreprocessing(dataset_dir, subset_dir, metadata_file, unwanted_metadata_file)


Created subset: C:\Users\isaac\Desktop\BigEarthTests\Subsets\50%
Created subset: C:\Users\isaac\Desktop\BigEarthTests\Subsets\10%
Created subset: C:\Users\isaac\Desktop\BigEarthTests\Subsets\1%


Processing folders: 100%|██████████| 115/115 [00:00<00:00, 5148.20it/s]


Total subfolder count in full dataset: 4750

Folder: 50%BigEarthNet | Subfolder Count: 2352 | Percentage: 49.52%
Folder: 10%BigEarthNet | Subfolder Count: 441 | Percentage: 9.28%
Folder: 1%BigEarthNet | Subfolder Count: 115 | Percentage: 2.42%
['Arable land' 'Broad-leaved forest' 'Mixed forest' 'Pastures'
 'Inland waters' 'Coniferous forest' 'Complex cultivation patterns'
 'Land principally occupied by agriculture, with significant areas of natural vegetation'
 'Urban fabric' 'Industrial or commercial units' 'Inland wetlands'
 'Transitional woodland, shrub'
 'Natural grassland and sparsely vegetated areas'
 'Moors, heathland and sclerophyllous vegetation' 'Marine waters'
 'Coastal wetlands' 'Permanent crops' 'Beaches, dunes, sands'
 'Agro-forestry areas']
Index(['patch_id', 'labels', 'split', 'country', 's1_name', 's2v1_name',
       'contains_seasonal_snow', 'contains_cloud_or_shadow', 'binary_vector'],
      dtype='object') 

Index(['patch_id', 'labels', 'split', 'country', 's1_name'

In [48]:
dataset_dir = r'C:\Users\isaac\Desktop\BigEarthTests\Subsets\1%Copy'

bands_of_interest = ['B01', 'B05', 'B06', 'B07', 'B8A', 'B09', 'B11', 'B12']

start_time = time.time()  # Start the timer

for folder in os.listdir(dataset_dir):
    folder_path = os.path.join(dataset_dir, folder)
    if os.path.isdir(folder_path):
        for subfolder in tqdm(os.listdir(folder_path), desc='Resizing TIFF files'):
            subfolder_path = os.path.join(folder_path, subfolder)
            if os.path.isdir(subfolder_path):
                for band in bands_of_interest:
                    band_source = subfolder_path + "/" + subfolder + "_" + band + ".tif"
                    temp_tif = subfolder_path + "/" + subfolder + "_" + band + "_resized.tif"
                    new_width = 120
                    new_height = 120

                    resizeTiffFiles(band_source, temp_tif, new_width, new_height)

                    os.remove(band_source)  # Delete the original
                    os.rename(temp_tif, band_source)  # Rename the temporary file

end_time = time.time()  # End the timer
execution_time = end_time - start_time  # Calculate the execution time

print(f"Execution time: {execution_time} seconds")

Resizing TIFF files: 100%|██████████| 1/1 [00:01<00:00,  1.19s/it]

Execution time: 133.3486819267273 seconds


In [9]:
row = metadata_df.loc[metadata_df['patch_id'] == 'S2A_MSIL2A_20171002T112111_N9999_R037_T29SNB_71_20']

labels = row['labels'].values[0]

print(f"Labels: {labels}")

Labels: ['Arable land', 'Land principally occupied by agriculture, with significant areas of natural vegetation']
